In [16]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import pandas as pd
import numpy as np
import pandas as pd
import os
import time
import datetime as dt

In [2]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
api_key = 'Place your own'
api_secret = 'Place your own'
client = Client(api_key, api_secret)

In [10]:
from datetime import date
from dateutil.relativedelta import relativedelta

today = date.today()

last_year = currentTimeDate = date.today() - relativedelta(years=1)
print("Today's date:", last_year)

Today's date: 2020-12-01


In [4]:
data_binance=cg.get_exchanges_by_id('binance')
df_binance =pd.DataFrame(data_binance['tickers'], columns=['base','target','volume','coin_id'])
df_binance.head()

,base,target,volume,coin_id
0,BUSD,USDT,1.104538e+09,binance-usd
1,ETH,USDT,7.359821e+05,ethereum
2,BTC,USDT,4.753089e+04,bitcoin
3,USDC,USDT,1.665523e+08,usd-coin
4,SHIB,USDT,4.819978e+13,shiba-inu


In [12]:
def get_data(from_d, to_d, crypto, vstoken):
    klines = client.get_historical_klines(crypto+vstoken, Client.KLINE_INTERVAL_1DAY, from_d, to_d)
    df = pd.DataFrame(klines)

    df.columns = ['open_time', 'o', 'h', 'l', 'c', 'v', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.open_time]
    df.index.name = 'datetime'
    df.drop(['open_time', 'close_time', 'ignore'], axis=1, inplace=True)
    df.drop(['qav', 'num_trades', 'taker_base_vol','taker_quote_vol'], axis=1, inplace=True)
    return df

In [14]:
def update_dataset(proj_name, vstoken):
    
    today = date.today()
    last_year = currentTimeDate = date.today() - relativedelta(years=1)
    today = str(today)
    last_year = str(last_year)


    filename = "data/"+proj_name+'.csv'

    if os.path.exists(filename):
        crypto_data = pd.read_csv(filename)  
        since_last = crypto_data.iloc[-1]['datetime']

        if since_last >= today:
            print('is up to date')
        else:
            print("updating")
            update = get_data(since_last, today, proj_name, vstoken)
            # we need to drop the first raw because it will be at double otherwise
            update = update.iloc[1: , :]
            crypto_data.append(update)
            update.to_csv(filename, mode='a', header=False)

    else:
        crypto_data = get_data(last_year, today, proj_name, vstoken)
        crypto_data.to_csv(filename)



In [17]:
for coin in df_binance.values:
    if(coin[1] == 'USDT'):
        update_dataset(coin[0],coin[1])
        print(coin[0])

updating
BUSD
updating
ETH
updating
BTC
updating
USDC
updating
SHIB
                                o             h             l             c  \
datetime                                                                      
2020-12-01 01:00:00    1.96410000    2.14040000    1.91680000    1.97380000   
2020-12-02 01:00:00    1.97020000    2.20980000    1.93330000    2.12090000   
2020-12-03 01:00:00    2.11900000    2.22790000    2.07400000    2.09670000   
2020-12-04 01:00:00    2.09670000    2.11230000    1.83600000    1.85350000   
2020-12-05 01:00:00    1.85070000    1.98300000    1.82000000    1.97950000   
...                           ...           ...           ...           ...   
2021-11-27 01:00:00  192.37000000  199.39000000  190.61000000  192.74000000   
2021-11-28 01:00:00  192.74000000  202.10000000  180.93000000  200.69000000   
2021-11-29 01:00:00  200.71000000  212.75000000  199.63000000  204.09000000   
2021-11-30 01:00:00  204.10000000  218.28000000  199.55000000  

CHZ
                               o            h            l            c  \
datetime                                                                  
2021-09-09 02:00:00   3.00000000  16.43500000   3.00000000  12.97100000   
2021-09-10 02:00:00  12.99200000  14.50000000   9.82100000  10.42500000   
2021-09-11 02:00:00  10.42500000  12.75900000  10.41400000  10.93300000   
2021-09-12 02:00:00  10.92900000  12.69100000  10.69200000  11.30300000   
2021-09-13 02:00:00  11.29300000  11.48400000   9.10000000  10.40000000   
...                          ...          ...          ...          ...   
2021-11-27 01:00:00  12.82100000  13.48800000  12.56500000  12.96700000   
2021-11-28 01:00:00  12.97700000  13.00200000  11.32000000  12.62300000   
2021-11-29 01:00:00  12.60400000  12.89900000  12.15500000  12.55500000   
2021-11-30 01:00:00  12.55400000  13.28600000  12.00000000  12.55400000   
2021-12-01 01:00:00  12.55900000  13.18400000  12.44200000  12.71200000   

                    